In [27]:
import numpy as np
import datasets
from scipy.optimize import minimize
from scipy.special import expit
from sklearn.metrics import log_loss
from softmax import softmax, log_softmax
import theano
from theano import tensor as T
import tensorflow as tf
from scipy import stats
from numpy.linalg import inv

In [2]:
X, y = datasets.htwt()
Y = datasets.one_hot(y)
N, D = X.shape
N, C = Y.shape
Xaug = np.hstack([np.ones((N, 1)), X])

In [3]:
z = T.dvector()
decode = lambda params: (params[1:].reshape((D, 1)), params[0])
mu = lambda w, b: T.nnet.sigmoid(T.dot(X, w) + b)
loss = T.sum(T.nnet.binary_crossentropy(mu(*decode(z)), y.reshape((N, 1))))
grad = theano.function([z], T.grad(loss, z))
hess = theano.function([z], T.hessian(loss, z))
loss = theano.function([z], loss)

In [4]:
mu = lambda w: expit(Xaug.dot(w))
loss = lambda w: log_loss(y, mu(w), normalize = False)
grad = lambda w: Xaug.T.dot(mu(w) - y)
S = lambda mu: np.diag(mu * (1 - mu))
hess = lambda w: Xaug.T.dot(S(mu(w))).dot(Xaug)

In [5]:
params = [0] * (D + 1)
w = minimize(loss, params).x
w = minimize(loss, params, method = 'Newton-CG', jac = grad, hess = hess, tol = 1e-6).x
yhat = expit(Xaug.dot(w)) > 0.5
print '%0.3f' % np.mean(yhat != y)

0.119


In [6]:
decode = lambda params: params.reshape((D + 1, C))
z = T.dvector()
mu = lambda W: T.nnet.softmax(T.dot(Xaug, W))
loss = T.sum(T.nnet.categorical_crossentropy(mu(decode(z)), Y))
grad = theano.function([z], T.grad(loss, z))
hess = theano.function([z], T.hessian(loss, z))    
loss = theano.function([z], loss)

In [7]:
decode = lambda params: np.array(params).reshape((D + 1, C))

mu = lambda W: softmax(Xaug.dot(W))
loss = lambda params: log_loss(y, mu(decode(params)), normalize = False)

def grad(params):
    Z = mu(decode(params)) - Y
    return sum([np.kron(Xaug[i], z) for i, z in enumerate(Z)])

def hess(params):
    Mu = mu(decode(params))
    o = lambda x: np.outer(x, x)
    return sum([np.kron(o(Xaug[i]), np.diag(z) - o(z)) for i, z in enumerate(Mu)])

In [8]:
params = [0] * (D + 1) * C
params = minimize(loss, params, method = 'Newton-CG', jac = grad, hess = hess, tol = 1e-6).x
W = params.reshape((D + 1, C))
yhat = np.argmax(softmax(Xaug.dot(W)), axis = 1)
print '%0.3f' % np.mean(yhat != y)

0.114


In [34]:
mvn = stats.multivariate_normal
log_prior = lambda w: mvn.logpdf(w[1:], np.zeros(D), 100 * np.eye(D))
mu = lambda w: expit(Xaug.dot(w))
nll = lambda w: log_loss(y, mu(w), normalize = False)
E = lambda w: nll(w) - log_prior(w)
params = np.zeros(D + 1)
w_hat = minimize(E, params).x
S = lambda mu: np.diag(mu * (1 - mu))
H = Xaug.T.dot(S(mu(w_hat))).dot(Xaug) + 1.0 / 200
posterior = mvn(w_hat, inv(H))
posterior.rvs()

array([-35.72143655,   0.40408765,   0.05280663])